# Milestone Project 1: Food Vision Big

See the annoted version of this notebook on GitHub: https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb

# Check GPU

Using mixed precision training we need a GPU with score 7.0+ (see here: https://developer.nvidia.com/cuda-gpus)

## Get helper functions 

In past modules, we've created a bunch of different helper functions to do small tasks required for our notebooks.

Rather than rewrite all of these, we can import a script and load them in from there.

The script we've got avialable can be found on GitHub: https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/helper_functions.py


In [3]:
# Download helper_function script
!wget -P ./Extras/ https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2024-09-28 15:38:59--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: './Extras/helper_functions.py'

     0K ..........                                            100% 2.56M=0.004s

2024-09-28 15:39:00 (2.56 MB/s) - './Extras/helper_functions.py' saved [10246/10246]



In [8]:
# import series of helper functions for the notebook
from Extras.helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys